## Date Handling of Input Files

> The data contains periods in CET/CEST format.
	

* we need to have start and end date in one consitent time zone
* we need to split the period into start and end date of the period 

"01/01/2015 00:00:00 - 01/01/2015 00:15:00"


## Setup

All the manipulations and plots in this notebook can be created with standard libraries such as matplotlib, statsmodels etc. 

In [31]:
# Main data packages. 
import numpy as np
import pandas as pd


## Import Data 

The data for this notebook was downloaded from the [meteoblue website](https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604) and consits of weather data for the city of Basel from 2008 till 2020. 

In [32]:
files = [
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_201412312300-201512312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_201512312300-201612312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_201612312300-201712312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_201712312300-201812312300 - 1.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_201712312300-201812312300 - 2.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_201812312300-201912312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_201912312300-202012312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_202012312300-202112312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_202112312300-202212312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_202212312300-202312312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_202312312300-202412312300.csv",
    "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_202412312300-202512312300.csv",
    # "../../data/res/solar/GUI_WIND_SOLAR_GENERATION_FORECAST_SOLAR_202512312300-202612312300.csv",
]

raw_df = pd.concat((pd.read_csv(f, delimiter=",") for f in files), ignore_index=True)


In [33]:
raw_df.shape  #(541275, 6)

(420768, 6)

In [34]:
raw_df.columns

Index(['MTU (CET/CEST)', 'Area', 'Day-ahead (MW)', 'Intraday (MW)',
       'Current (MW)', 'Actual (MW)'],
      dtype='object')

In [35]:
# Create working copy of dataframe
df_utc_op = raw_df.copy()
df_utc_op = df_utc_op[["MTU (CET/CEST)","Day-ahead (MW)", "Actual (MW)"]].rename(  #"Intraday (MW)","Current (MW)",
    columns={
        'MTU (CET/CEST)': 'period',  
        "Day-ahead (MW)": "solar_da",
        # "Intraday (MW)": "OffSh Intraday (MW)",
        # "Current (MW)": "OffSh Current (MW)",
        "Actual (MW)": "solar_act",
    }
)
df_utc_op.head()

,period,solar_da,solar_act
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,0.0,0.15
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,0.0,0.15
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,0.0,0.15
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,0.0,0.15
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,0.0,0.16


In [36]:
df_utc_op.shape

(420768, 3)

In [37]:
# Split "period" into start/end strings
df_utc_op[["period_start", "period_end"]] = df_utc_op["period"].str.split(" - ", n=1, expand=True)
df_utc_op.head()

,period,solar_da,solar_act,period_start,period_end
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,0.0,0.15,01/01/2015 00:00:00,01/01/2015 00:15:00
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,0.0,0.15,01/01/2015 00:15:00,01/01/2015 00:30:00
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,0.0,0.15,01/01/2015 00:30:00,01/01/2015 00:45:00
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,0.0,0.15,01/01/2015 00:45:00,01/01/2015 01:00:00
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,0.0,0.16,01/01/2015 01:00:00,01/01/2015 01:15:00


In [38]:

def add_timezone_and_utc(df, col): #="period_start"):
    # State: start in CET
    tz_state = "CET"
    tz_list = []

    # Walk rows in order and flip state if original string contains CET/CEST
    for val in df[col].astype(str):
        if "CEST" in val:
            tz_state = "CEST"
        elif "CET" in val:
            tz_state = "CET"
        tz_list.append(tz_state)

    # Parse datetime from the column (remove any existing timezone text)
    base_dt = pd.to_datetime(
        df[col].astype(str).str.replace(r"\s*\(.*\)$", "", regex=True),
        dayfirst=True,
        errors="coerce"
    )

    # Add labeled version
    # df["period_start_labeled"] = (
    df[col + "_labeled"] = (
        base_dt.dt.strftime("%d/%m/%Y %H:%M:%S") + " (" + pd.Series(tz_list, index=df.index) + ")"
    )

    # Compute UTC by subtracting 1 or 2 hours depending on CET/CEST
    offset_hours = pd.Series(tz_list, index=df.index).map({"CET": 1, "CEST": 2})
    df[col + "_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")
    # df["tsp_start_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")

    return df


In [39]:
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_start")
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_end")

In [40]:
df_utc_op

,period,solar_da,solar_act,period_start,period_end,period_start_labeled,period_start_utc,period_end_labeled,period_end_utc
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,0.0,0.15,01/01/2015 00:00:00,01/01/2015 00:15:00,01/01/2015 00:00:00 (CET),2014-12-31 23:00:00,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,0.0,0.15,01/01/2015 00:15:00,01/01/2015 00:30:00,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,0.0,0.15,01/01/2015 00:30:00,01/01/2015 00:45:00,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,0.0,0.15,01/01/2015 00:45:00,01/01/2015 01:00:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,0.0,0.16,01/01/2015 01:00:00,01/01/2015 01:15:00,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00,01/01/2015 01:15:00 (CET),2015-01-01 00:15:00
...,...,...,...,...,...,...,...,...,...
420763,31/12/2025 22:45:00 - 31/12/2025 23:00:00,0.0,0.04,31/12/2025 22:45:00,31/12/2025 23:00:00,31/12/2025 22:45:00 (CET),2025-12-31 21:45:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00
420764,31/12/2025 23:00:00 - 31/12/2025 23:15:00,0.0,0.03,31/12/2025 23:00:00,31/12/2025 23:15:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00
420765,31/12/2025 23:15:00 - 31/12/2025 23:30:00,0.0,0.04,31/12/2025 23:15:00,31/12/2025 23:30:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00
420766,31/12/2025 23:30:00 - 31/12/2025 23:45:00,0.0,0.05,31/12/2025 23:30:00,31/12/2025 23:45:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00,31/12/2025 23:45:00 (CET),2025-12-31 22:45:00


In [41]:
df_utc_op.columns

Index(['period', 'solar_da', 'solar_act', 'period_start', 'period_end',
       'period_start_labeled', 'period_start_utc', 'period_end_labeled',
       'period_end_utc'],
      dtype='object')

In [42]:
# Create working copy of dataframe
df_utc_op_temp = df_utc_op.copy()
# select only those values
# df_price = df_price[df_price["Sequence"] != "Sequence Sequence 2"]
 # drop columns
df_utc_op.drop(columns=['period_start', 'period_end', 'period_start_labeled', 'period_end_labeled'], inplace=True)

df_utc_op = df_utc_op.assign(
    date1=lambda x: x["period_start_utc"].dt.date,
    date=lambda x: pd.to_datetime(
        x["date1"],
        format="mixed",
        dayfirst=True,
        errors="coerce"
    ),
    year=lambda x: x["period_start_utc"].dt.year,
    month=lambda x: x["period_start_utc"].dt.month,
    #monthname=lambda x: x["period_start_utc"].dt.month_name(),
    day=lambda x: x["period_start_utc"].dt.day,
    dayofyear=lambda x: x["period_start_utc"].dt.dayofyear,
    hour=lambda x: x["period_start_utc"].dt.hour,
    #week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype(int),
    week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype("Int64"),
    dayofweek=lambda x: x["period_start_utc"].dt.dayofweek,   # 0=Mon ... 6=Sun
    #dayname=lambda x: x["period_start_utc"].dt.day_name(),   

)
df_utc_op.drop(columns=['date1'], inplace=True)

df_utc_op.head()

,period,solar_da,solar_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,0.0,0.15,2014-12-31 23:00:00,2014-12-31 23:15:00,2014-12-31,2014,12,31,365,23,1,2
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,0.0,0.15,2014-12-31 23:15:00,2014-12-31 23:30:00,2014-12-31,2014,12,31,365,23,1,2
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,0.0,0.15,2014-12-31 23:30:00,2014-12-31 23:45:00,2014-12-31,2014,12,31,365,23,1,2
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,0.0,0.15,2014-12-31 23:45:00,2015-01-01 00:00:00,2014-12-31,2014,12,31,365,23,1,2
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,0.0,0.16,2015-01-01 00:00:00,2015-01-01 00:15:00,2015-01-01,2015,1,1,1,0,1,3


In [43]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            420768 non-null  object        
 1   solar_da          420756 non-null  object        
 2   solar_act         420575 non-null  object        
 3   period_start_utc  420768 non-null  datetime64[ns]
 4   period_end_utc    420768 non-null  datetime64[ns]
 5   date              420768 non-null  datetime64[ns]
 6   year              420768 non-null  int32         
 7   month             420768 non-null  int32         
 8   day               420768 non-null  int32         
 9   dayofyear         420768 non-null  int32         
 10  hour              420768 non-null  int32         
 11  week              420768 non-null  Int64         
 12  dayofweek         420768 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), int32(6), object(3)
memory 

In [44]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["solar_da"].isna()]
rows_with_missing_price.tail(100)

,period,solar_da,solar_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
344444,29/10/2023 00:00:00 - 29/10/2023 00:15:00,NaN,26.28,2023-10-28 22:00:00,2023-10-28 22:15:00,2023-10-28,2023,10,28,301,22,43,5
344445,29/10/2023 00:15:00 - 29/10/2023 00:30:00,NaN,26.17,2023-10-28 22:15:00,2023-10-28 22:30:00,2023-10-28,2023,10,28,301,22,43,5
344446,29/10/2023 00:30:00 - 29/10/2023 00:45:00,NaN,2.64,2023-10-28 22:30:00,2023-10-28 22:45:00,2023-10-28,2023,10,28,301,22,43,5
344447,29/10/2023 00:45:00 - 29/10/2023 01:00:00,NaN,2.09,2023-10-28 22:45:00,2023-10-28 23:00:00,2023-10-28,2023,10,28,301,22,43,5
379388,27/10/2024 00:00:00 - 27/10/2024 00:15:00,NaN,26.13,2024-10-26 22:00:00,2024-10-26 22:15:00,2024-10-26,2024,10,26,300,22,43,5
379389,27/10/2024 00:15:00 - 27/10/2024 00:30:00,NaN,27.7,2024-10-26 22:15:00,2024-10-26 22:30:00,2024-10-26,2024,10,26,300,22,43,5
379390,27/10/2024 00:30:00 - 27/10/2024 00:45:00,NaN,18.56,2024-10-26 22:30:00,2024-10-26 22:45:00,2024-10-26,2024,10,26,300,22,43,5
379391,27/10/2024 00:45:00 - 27/10/2024 01:00:00,NaN,19.24,2024-10-26 22:45:00,2024-10-26 23:00:00,2024-10-26,2024,10,26,300,22,43,5
414332,26/10/2025 00:00:00 - 26/10/2025 00:15:00,NaN,22.97,2025-10-25 22:00:00,2025-10-25 22:15:00,2025-10-25,2025,10,25,298,22,43,5
414333,26/10/2025 00:15:00 - 26/10/2025 00:30:00,NaN,21.8,2025-10-25 22:15:00,2025-10-25 22:30:00,2025-10-25,2025,10,25,298,22,43,5


In [45]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["solar_act"].isna()]
rows_with_missing_price

,period,solar_da,solar_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
49624,31/05/2016 23:00:00 - 31/05/2016 23:15:00,0.0,NaN,2016-05-31 21:00:00,2016-05-31 21:15:00,2016-05-31,2016,5,31,152,21,22,1
49625,31/05/2016 23:15:00 - 31/05/2016 23:30:00,0.0,NaN,2016-05-31 21:15:00,2016-05-31 21:30:00,2016-05-31,2016,5,31,152,21,22,1
49626,31/05/2016 23:30:00 - 31/05/2016 23:45:00,0.0,NaN,2016-05-31 21:30:00,2016-05-31 21:45:00,2016-05-31,2016,5,31,152,21,22,1
49627,31/05/2016 23:45:00 - 01/06/2016 00:00:00,0.0,NaN,2016-05-31 21:45:00,2016-05-31 22:00:00,2016-05-31,2016,5,31,152,21,22,1
49628,01/06/2016 00:00:00 - 01/06/2016 00:15:00,0.0,NaN,2016-05-31 22:00:00,2016-05-31 22:15:00,2016-05-31,2016,5,31,152,22,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64024,28/10/2016 23:00:00 - 28/10/2016 23:15:00,0.0,NaN,2016-10-28 21:00:00,2016-10-28 21:15:00,2016-10-28,2016,10,28,302,21,43,4
64025,28/10/2016 23:15:00 - 28/10/2016 23:30:00,0.0,NaN,2016-10-28 21:15:00,2016-10-28 21:30:00,2016-10-28,2016,10,28,302,21,43,4
64026,28/10/2016 23:30:00 - 28/10/2016 23:45:00,0.0,NaN,2016-10-28 21:30:00,2016-10-28 21:45:00,2016-10-28,2016,10,28,302,21,43,4
64027,28/10/2016 23:45:00 - 29/10/2016 00:00:00,0.0,NaN,2016-10-28 21:45:00,2016-10-28 22:00:00,2016-10-28,2016,10,28,302,21,43,4


In [46]:
# Convert to numeric, turn non-numeric strings into NaN
df_utc_op["solar_act"] = pd.to_numeric(df_utc_op["solar_act"], errors="coerce")
df_utc_op["solar_da"] = pd.to_numeric(df_utc_op["solar_da"], errors="coerce")

rows_with_missing_price = df_utc_op[df_utc_op["solar_act"].isna()]
rows_with_missing_price

,period,solar_da,solar_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
49624,31/05/2016 23:00:00 - 31/05/2016 23:15:00,0.00,NaN,2016-05-31 21:00:00,2016-05-31 21:15:00,2016-05-31,2016,5,31,152,21,22,1
49625,31/05/2016 23:15:00 - 31/05/2016 23:30:00,0.00,NaN,2016-05-31 21:15:00,2016-05-31 21:30:00,2016-05-31,2016,5,31,152,21,22,1
49626,31/05/2016 23:30:00 - 31/05/2016 23:45:00,0.00,NaN,2016-05-31 21:30:00,2016-05-31 21:45:00,2016-05-31,2016,5,31,152,21,22,1
49627,31/05/2016 23:45:00 - 01/06/2016 00:00:00,0.00,NaN,2016-05-31 21:45:00,2016-05-31 22:00:00,2016-05-31,2016,5,31,152,21,22,1
49628,01/06/2016 00:00:00 - 01/06/2016 00:15:00,0.00,NaN,2016-05-31 22:00:00,2016-05-31 22:15:00,2016-05-31,2016,5,31,152,22,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166456,30/09/2018 23:00:00 - 30/09/2018 23:15:00,NaN,NaN,2018-09-30 21:00:00,2018-09-30 21:15:00,2018-09-30,2018,9,30,273,21,39,6
166457,30/09/2018 23:15:00 - 30/09/2018 23:30:00,NaN,NaN,2018-09-30 21:15:00,2018-09-30 21:30:00,2018-09-30,2018,9,30,273,21,39,6
166458,30/09/2018 23:30:00 - 30/09/2018 23:45:00,NaN,NaN,2018-09-30 21:30:00,2018-09-30 21:45:00,2018-09-30,2018,9,30,273,21,39,6
166459,30/09/2018 23:45:00 - 01/10/2018 00:00:00,NaN,NaN,2018-09-30 21:45:00,2018-09-30 22:00:00,2018-09-30,2018,9,30,273,21,39,6


In [47]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            420768 non-null  object        
 1   solar_da          385716 non-null  float64       
 2   solar_act         385535 non-null  float64       
 3   period_start_utc  420768 non-null  datetime64[ns]
 4   period_end_utc    420768 non-null  datetime64[ns]
 5   date              420768 non-null  datetime64[ns]
 6   year              420768 non-null  int32         
 7   month             420768 non-null  int32         
 8   day               420768 non-null  int32         
 9   dayofyear         420768 non-null  int32         
 10  hour              420768 non-null  int32         
 11  week              420768 non-null  Int64         
 12  dayofweek         420768 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), float64(2), int32(6), objec

In [48]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "hour", "week", "dayofweek"
]

df_utc_h = (
    df_utc_op
    .groupby(group_cols, as_index=False)
    .agg(
        solar_da=("solar_da", "mean"),
        solar_act=("solar_act", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_hour=("year", "size"),   # count rows per group
    )
)
df_utc_h.head()

,date,year,month,day,dayofyear,hour,week,dayofweek,solar_da,solar_act,period_start_utc,period_end_utc,c_by_hour
0,2014-12-31,2014,12,31,365,23,1,2,0.0,0.1500,2014-12-31 23:00:00,2015-01-01 00:00:00,4
1,2015-01-01,2015,1,1,1,0,1,3,0.0,0.1525,2015-01-01 00:00:00,2015-01-01 01:00:00,4
2,2015-01-01,2015,1,1,1,1,1,3,0.0,0.1500,2015-01-01 01:00:00,2015-01-01 02:00:00,4
3,2015-01-01,2015,1,1,1,2,1,3,0.0,0.1525,2015-01-01 02:00:00,2015-01-01 03:00:00,4
4,2015-01-01,2015,1,1,1,3,1,3,0.0,0.1425,2015-01-01 03:00:00,2015-01-01 04:00:00,4


In [49]:
df_utc_h.tail()

,date,year,month,day,dayofyear,hour,week,dayofweek,solar_da,solar_act,period_start_utc,period_end_utc,c_by_hour
96427,2025-12-31,2025,12,31,365,18,1,2,0.0,0.0675,2025-12-31 18:00:00,2025-12-31 19:00:00,4
96428,2025-12-31,2025,12,31,365,19,1,2,0.0,0.0525,2025-12-31 19:00:00,2025-12-31 20:00:00,4
96429,2025-12-31,2025,12,31,365,20,1,2,0.0,0.0575,2025-12-31 20:00:00,2025-12-31 21:00:00,4
96430,2025-12-31,2025,12,31,365,21,1,2,0.0,0.0400,2025-12-31 21:00:00,2025-12-31 22:00:00,4
96431,2025-12-31,2025,12,31,365,22,1,2,0.0,0.0450,2025-12-31 22:00:00,2025-12-31 23:00:00,4


In [50]:
df_utc_h["c_by_hour"].value_counts()

c_by_hour
4    87672
8     8760
Name: count, dtype: int64

In [51]:
df_utc_h[df_utc_h["c_by_hour"] > 4]

,date,year,month,day,dayofyear,hour,week,dayofweek,solar_da,solar_act,period_start_utc,period_end_utc,c_by_hour
26304,2017-12-31,2017,12,31,365,23,52,6,0.0,0.3850,2017-12-31 23:00:00,2018-01-01 00:00:00,8
26305,2018-01-01,2018,1,1,1,0,1,0,0.0,0.3850,2018-01-01 00:00:00,2018-01-01 01:00:00,8
26306,2018-01-01,2018,1,1,1,1,1,0,0.0,0.3825,2018-01-01 01:00:00,2018-01-01 02:00:00,8
26307,2018-01-01,2018,1,1,1,2,1,0,0.0,0.3925,2018-01-01 02:00:00,2018-01-01 03:00:00,8
26308,2018-01-01,2018,1,1,1,3,1,0,0.0,0.4050,2018-01-01 03:00:00,2018-01-01 04:00:00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31,2018,12,31,365,18,1,0,0.0,0.6200,2018-12-31 18:00:00,2018-12-31 19:00:00,8
35060,2018-12-31,2018,12,31,365,19,1,0,0.0,0.6075,2018-12-31 19:00:00,2018-12-31 20:00:00,8
35061,2018-12-31,2018,12,31,365,20,1,0,0.0,0.6100,2018-12-31 20:00:00,2018-12-31 21:00:00,8
35062,2018-12-31,2018,12,31,365,21,1,0,0.0,0.6175,2018-12-31 21:00:00,2018-12-31 22:00:00,8


In [52]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "week", "dayofweek"
]

df_utc_d = (
    df_utc_h
    .groupby(group_cols, as_index=False)
    .agg(
        solar_da=("solar_da", "mean"),
        solar_act=("solar_act", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_day=("year", "size"),   # count rows per group
    )
)
df_utc_d.head()

,date,year,month,day,dayofyear,week,dayofweek,solar_da,solar_act,period_start_utc,period_end_utc,c_by_day
0,2014-12-31,2014,12,31,365,1,2,0.000000,0.150000,2014-12-31 23:00:00,2015-01-01,1
1,2015-01-01,2015,1,1,1,1,3,626.756875,691.415625,2015-01-01 00:00:00,2015-01-02,24
2,2015-01-02,2015,1,2,2,1,4,239.736458,332.239688,2015-01-02 00:00:00,2015-01-03,24
3,2015-01-03,2015,1,3,3,1,5,265.695937,323.666042,2015-01-03 00:00:00,2015-01-04,24
4,2015-01-04,2015,1,4,4,1,6,813.873229,816.998229,2015-01-04 00:00:00,2015-01-05,24


In [53]:
df_utc_d.tail()

,date,year,month,day,dayofyear,week,dayofweek,solar_da,solar_act,period_start_utc,period_end_utc,c_by_day
4014,2025-12-27,2025,12,27,361,52,5,3018.103750,3252.567188,2025-12-27,2025-12-28 00:00:00,24
4015,2025-12-28,2025,12,28,362,52,6,3509.610625,3575.586354,2025-12-28,2025-12-29 00:00:00,24
4016,2025-12-29,2025,12,29,363,1,0,2167.589271,2121.160625,2025-12-29,2025-12-30 00:00:00,24
4017,2025-12-30,2025,12,30,364,1,1,3420.363854,3245.597188,2025-12-30,2025-12-31 00:00:00,24
4018,2025-12-31,2025,12,31,365,1,2,1692.922935,1965.222826,2025-12-31,2025-12-31 23:00:00,23


In [54]:
df_utc_d["c_by_day"].value_counts()

c_by_day
24    4017
1        1
23       1
Name: count, dtype: int64

In [55]:
group_cols = [
    "year"
]

df_utc_y = (
    df_utc_d
    .groupby(group_cols, as_index=False)
    .agg(
        solar_da=("solar_da", "mean"),
        solar_act=("solar_act", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_year=("year", "size"),   # count rows per group
    )
)
df_utc_y.head(20)

,year,solar_da,solar_act,period_start_utc,period_end_utc,c_by_year
0,2014,0.000000,0.150000,2014-12-31 23:00:00,2015-01-01 00:00:00,1
1,2015,4163.000116,4065.505781,2015-01-01 00:00:00,2016-01-01 00:00:00,365
2,2016,4111.006682,4088.776989,2016-01-01 00:00:00,2017-01-01 00:00:00,366
3,2017,4211.140771,4227.561892,2017-01-01 00:00:00,2018-01-01 00:00:00,365
4,2018,4863.167672,4855.615190,2018-01-01 00:00:00,2019-01-01 00:00:00,365
5,2019,4745.097587,4775.466469,2019-01-01 00:00:00,2020-01-01 00:00:00,365
6,2020,5221.565365,5229.671971,2020-01-01 00:00:00,2021-01-01 00:00:00,366
7,2021,5330.379388,5299.324147,2021-01-01 00:00:00,2022-01-01 00:00:00,365
8,2022,6366.042900,6391.267751,2022-01-01 00:00:00,2023-01-01 00:00:00,365
9,2023,6386.776965,6369.718446,2023-01-01 00:00:00,2024-01-01 00:00:00,365


In [56]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["solar_da"].isna()]
rows_with_missing_price

,period,solar_da,solar_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
131420,01/10/2018 00:00:00 - 01/10/2018 00:15:00,NaN,NaN,2018-09-30 22:00:00,2018-09-30 22:15:00,2018-09-30,2018,9,30,273,22,39,6
131421,01/10/2018 00:15:00 - 01/10/2018 00:30:00,NaN,NaN,2018-09-30 22:15:00,2018-09-30 22:30:00,2018-09-30,2018,9,30,273,22,39,6
131422,01/10/2018 00:30:00 - 01/10/2018 00:45:00,NaN,NaN,2018-09-30 22:30:00,2018-09-30 22:45:00,2018-09-30,2018,9,30,273,22,39,6
131423,01/10/2018 00:45:00 - 01/10/2018 01:00:00,NaN,NaN,2018-09-30 22:45:00,2018-09-30 23:00:00,2018-09-30,2018,9,30,273,22,39,6
131424,01/10/2018 01:00:00 - 01/10/2018 01:15:00,NaN,NaN,2018-09-30 23:00:00,2018-09-30 23:15:00,2018-09-30,2018,9,30,273,23,39,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379391,27/10/2024 00:45:00 - 27/10/2024 01:00:00,NaN,19.24,2024-10-26 22:45:00,2024-10-26 23:00:00,2024-10-26,2024,10,26,300,22,43,5
414332,26/10/2025 00:00:00 - 26/10/2025 00:15:00,NaN,22.97,2025-10-25 22:00:00,2025-10-25 22:15:00,2025-10-25,2025,10,25,298,22,43,5
414333,26/10/2025 00:15:00 - 26/10/2025 00:30:00,NaN,21.80,2025-10-25 22:15:00,2025-10-25 22:30:00,2025-10-25,2025,10,25,298,22,43,5
414334,26/10/2025 00:30:00 - 26/10/2025 00:45:00,NaN,18.70,2025-10-25 22:30:00,2025-10-25 22:45:00,2025-10-25,2025,10,25,298,22,43,5


In [57]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["solar_act"].isna()]
rows_with_missing_price

,period,solar_da,solar_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
49624,31/05/2016 23:00:00 - 31/05/2016 23:15:00,0.00,NaN,2016-05-31 21:00:00,2016-05-31 21:15:00,2016-05-31,2016,5,31,152,21,22,1
49625,31/05/2016 23:15:00 - 31/05/2016 23:30:00,0.00,NaN,2016-05-31 21:15:00,2016-05-31 21:30:00,2016-05-31,2016,5,31,152,21,22,1
49626,31/05/2016 23:30:00 - 31/05/2016 23:45:00,0.00,NaN,2016-05-31 21:30:00,2016-05-31 21:45:00,2016-05-31,2016,5,31,152,21,22,1
49627,31/05/2016 23:45:00 - 01/06/2016 00:00:00,0.00,NaN,2016-05-31 21:45:00,2016-05-31 22:00:00,2016-05-31,2016,5,31,152,21,22,1
49628,01/06/2016 00:00:00 - 01/06/2016 00:15:00,0.00,NaN,2016-05-31 22:00:00,2016-05-31 22:15:00,2016-05-31,2016,5,31,152,22,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166456,30/09/2018 23:00:00 - 30/09/2018 23:15:00,NaN,NaN,2018-09-30 21:00:00,2018-09-30 21:15:00,2018-09-30,2018,9,30,273,21,39,6
166457,30/09/2018 23:15:00 - 30/09/2018 23:30:00,NaN,NaN,2018-09-30 21:15:00,2018-09-30 21:30:00,2018-09-30,2018,9,30,273,21,39,6
166458,30/09/2018 23:30:00 - 30/09/2018 23:45:00,NaN,NaN,2018-09-30 21:30:00,2018-09-30 21:45:00,2018-09-30,2018,9,30,273,21,39,6
166459,30/09/2018 23:45:00 - 01/10/2018 00:00:00,NaN,NaN,2018-09-30 21:45:00,2018-09-30 22:00:00,2018-09-30,2018,9,30,273,21,39,6


## saving our files by hour

In [58]:
df_utc_h.to_csv("../../data_cleaned/by_source/05_RES_SOLAR.csv", index=False)